# DB

In [37]:
import os, json

testydata: str = "./../../../testymolo/media/data/"
Proteins:list = json.load(open(os.path.join(testydata,'Protein.temp.json')))



## Lexicon

In [38]:
class Lexicon(): 

    content:dict = {
            'methyltransferase':["methyltransferase", "nsp16", "2'-O-methyltransferase", 'Mtase']
        }

    @staticmethod
    def get_synonym(q):
        for syn, values in Lexicon.content.items():
            if q in values:
                return syn
        return None
    
    

In [41]:
with open(os.path.join(testydata, "Lexicon.json"), 'w') as handle :
    handle.write(json.dumps(Lexicon.content))

# Input

In [39]:
#suq:str = input("BALLS ???")
#print(suq)
suq:str = "methyltransferase"

# Search

### aligning words

In [106]:
referenced_word = "AIBV[Beaudette]"
input_kw = "IBV"

spaces = '_'  ### default: ' ' \s
small_words:bool = True
## local alignment

ident:int = 2
subst:int
indel:int


if small_words:
    indel = max(-1, -2)
    subst = min(-1, -2)
else:
    indel = min(-1, -2)
    subst = max(-1, -2)

table:list = [[0] * (len(referenced_word)+1)] * (len(input_kw)+1) 
vectors:list = [[0] * len(referenced_word)] * len(input_kw)

In [107]:
print(*table, sep='\n')
print('\n')
print(*vectors, sep='\n')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [108]:
### filling
    # for each x,y where( x:range(1, len(reference)+1) and  y:range(1, len(input)+1)
        # calc score -> table(x,y)
        # fix vector -> vectors(x-1, y-1)

#  subst      -> 0        |ident/subst|  | insertion |  |  deletion |  |    y,x    |
#  insertion  -> 1        | +0  |  0  |  |  0  |  0  |  |  0  | +0  |  |-1,-1| 0,-1|
#  deletion   -> 2        |  0  | -M  |  |  +0 | -DI |  |  0  | -DI |  |-1, 0| 0, 0|

In [109]:
for y in range(1, len(input_kw)+1):
    for x in range(1, len(referenced_word)+1):
        print(x,y , '->', x-1, y-1,  '->', referenced_word[x-1], input_kw[y-1])
        if referenced_word[x-1].lower() == input_kw[y-1].lower():
            table[y][x] = table[y-1][x-1] + ident
            vectors[y-1][x-1] = 1
        else:
            costs:tuple = ((table[y-1][x-1] + subst), (table[y-1][x] + indel), (table[y][x-1] + indel))
            minimum:int = min(costs)
            k:int = costs.index(minimum)
            #
            table[y][x] = minimum
            vectors[y-1][x-1] = k

1 1 -> 0 0 -> A I
2 1 -> 1 0 -> I I
3 1 -> 2 0 -> B I
4 1 -> 3 0 -> V I
5 1 -> 4 0 -> [ I
6 1 -> 5 0 -> B I
7 1 -> 6 0 -> e I
8 1 -> 7 0 -> a I
9 1 -> 8 0 -> u I
10 1 -> 9 0 -> d I
11 1 -> 10 0 -> e I
12 1 -> 11 0 -> t I
13 1 -> 12 0 -> t I
14 1 -> 13 0 -> e I
15 1 -> 14 0 -> ] I
1 2 -> 0 1 -> A B
2 2 -> 1 1 -> I B
3 2 -> 2 1 -> B B
4 2 -> 3 1 -> V B
5 2 -> 4 1 -> [ B
6 2 -> 5 1 -> B B
7 2 -> 6 1 -> e B
8 2 -> 7 1 -> a B
9 2 -> 8 1 -> u B
10 2 -> 9 1 -> d B
11 2 -> 10 1 -> e B
12 2 -> 11 1 -> t B
13 2 -> 12 1 -> t B
14 2 -> 13 1 -> e B
15 2 -> 14 1 -> ] B
1 3 -> 0 2 -> A V
2 3 -> 1 2 -> I V
3 3 -> 2 2 -> B V
4 3 -> 3 2 -> V V
5 3 -> 4 2 -> [ V
6 3 -> 5 2 -> B V
7 3 -> 6 2 -> e V
8 3 -> 7 2 -> a V
9 3 -> 8 2 -> u V
10 3 -> 9 2 -> d V
11 3 -> 10 2 -> e V
12 3 -> 11 2 -> t V
13 3 -> 12 2 -> t V
14 3 -> 13 2 -> e V
15 3 -> 14 2 -> ] V


In [110]:
print(*table, sep='\n')
print('\n')
print(*vectors, sep='\n')

[0, -4, -6, -8, -6, -8, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28]
[0, -4, -6, -8, -6, -8, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28]
[0, -4, -6, -8, -6, -8, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28]
[0, -4, -6, -8, -6, -8, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28]


[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
